Let's train the guy

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm as tqdm
tf.keras.backend.set_floatx('float32')
from collections import deque
from datetime import datetime
import random
import matplotlib
from environment import Environment
from plots import just_plot
from misc import *
from nets import *
from buffer import ReplayBuffer


amplitude=0.4
tau = 0.1
lr_critic = 0.005
lr_actor=10**-3
noise_displacement = .25
ep_guess=0.01
dolinar_layers=1
number_phases=2
buffer_size = 10**7


env = Environment(amplitude=amplitude, dolinar_layers = dolinar_layers, number_phases=number_phases)
buffer = ReplayBuffer(buffer_size=buffer_size)

critic = Critic(nature="primary",valreg=0.0, dolinar_layers = dolinar_layers, number_phases=number_phases)
critic_target = Critic(nature="target", valreg=0.0,dolinar_layers = dolinar_layers, number_phases=number_phases, tau = tau)
actor = Actor(nature="primary",valreg=0.0, dolinar_layers = dolinar_layers)
actor_target = Actor(nature="target",valreg=0.0, dolinar_layers = dolinar_layers)

optimizer_critic = tf.keras.optimizers.Adam(lr=lr_critic)
optimizer_actor = tf.keras.optimizers.SGD(lr=lr_actor)

policy_evaluator = PolicyEvaluator(amplitude = amplitude, dolinar_layers=dolinar_layers, number_phases = number_phases)

expes = np.load("buffers/1L-stoch.npy")

for k in tqdm(expes):
    buffer.add(tuple(k))
    
    
def plot(critic, episode = 1, last_episode=False, max_episode=10**4, history_betas =[]):

    betas = np.arange(.1,1.1,.05)
    inps = np.stack([np.ones(len(betas))*critic.pad_value, betas], axis=1)
    inps = np.reshape(inps, (len(betas),1,2))
    ax3.plot(betas, np.squeeze(critic(inps)), alpha=min(episode/max_episode,1), linewidth=5,label="RNN - "+str(episode))

    for outcome in [0.,1.]:
       for guess_index in [0.,1.]:
            m=[]
            for k in tf.unstack(inps):
                m.append(tf.concat([k, np.reshape(np.array([outcome,guess_index]), (1,2))], axis=0))
            axes[str(outcome)].plot(betas, np.squeeze(critic(tf.stack(m, axis=0)))[:,1], alpha=min(episode/max_episode,1), linewidth=5,label="RNN" +str(episode))
            
            
    if last_episode:
        betas = np.arange(.1,1.1,.05)
        inps = np.stack([np.ones(len(betas))*critic.pad_value, betas], axis=1)
        inps = np.reshape(inps, (len(betas),1,2))
        ax3.plot(betas, np.squeeze(critic(inps)), linewidth=8,c="black",label="RNN")


        for outcome in [0.,1.]:
           for guess_index in [0.,1.]:
                m=[]
                for k in tf.unstack(inps):
                    m.append(tf.concat([k, np.reshape(np.array([outcome,guess_index]), (1,2))], axis=0))
                axes[str(outcome)].plot(betas, np.squeeze(critic(tf.stack(m, axis=0)))[:,1],c="black", linewidth=8,label="RNN")


        ax1.plot(betas,[qval(b, 0, -1) for b in betas],c="red", linewidth=5, label="Q(n1=0,"+r'$\beta$'+"; g=-1)")
        ax1.plot(betas,[qval(b, 0, 1) for b in betas],c="blue",  linewidth=5,label="Q(n1=0,"+r'$\beta$'+"; g=1)")

        ax2.plot(betas,[qval(b, 1, -1) for b in betas],c="red", linewidth=5, label="Q(n1=0,"+r'$\beta$'+"; g=-1)")
        ax2.plot(betas,[qval(b, 1, 1) for b in betas],c="blue",  linewidth=5,label="Q(n1=0,"+r'$\beta$'+"; g=1)")

        ax3.plot(betas,ps_maxlik(betas), '--', linewidth=5, color="red", label="P*")
        ax4.plot(np.arange(1,len(history_betas)+1), history_betas, alpha=0.75, c="red", linewidth=7)
        for ax in [ax1, ax2, ax3]:
            ax.set_xlabel(r'$\beta$', size=20)


In [16]:
batch_size = 32.

experiences = buffer.sample(batch_size).astype(np.float32)
dq_da = critic_grad_tf(critic, experiences)
context_outcome_actor = np.reshape(np.array([actor.pad_value]),(1,1,1)).astype(np.float32)
actor.reset_states_workaround(new_batch_size=int(batch_size))


In [84]:

def monster_gradient(actor, dq_da, experiences, optimizer_actor):
    actor.lstm.stateful=False
    actor.reset_states_workaround(new_batch_size=experiences.shape[0])
    finns = [tf.ones((experiences.shape[0], 1,1))*actor.pad_value]
    unstacked_exp = tf.unstack(experiences, axis=1)
    for index in range(1,2*actor.dolinar_layers-2,2):
        finns.append(tf.reshape(unstacked_exp[index], (experiences.shape[0], 1,1)))
    final_preds = tf.concat(finns, axis=1)

    with tf.GradientTape() as tape:
        tape.watch(actor.trainable_variables)
        final_predss = actor(final_preds)
            
    da_dtheta1 = tape.jacobian(final_predss, actor.trainable_variables)

    variables = actor.trainable_variables
    grad_j=[]
    for k in range(len(da_dtheta1)):
        onnns = tf.ones(len(tf.shape(da_dtheta1[k]))-1)
        multip = tf.reshape(dq_da,tf.cast(tf.concat([tf.Variable([batch_size]),onnns], axis=0), np.int32))*da_dtheta1[k]
        multip = tf.reshape(tf.math.reduce_mean(multip, axis=0), variables[k].shape)
        grad_j.append(multip)
    #optimizer_actor.apply_gradients(zip(grad_j, actor.trainable_variables))
    return grad_j


In [85]:

def monster_gradient0(actor, dq_da, experiences, optimizer_actor):
    actor.lstm.stateful=False
    actor.reset_states_workaround(new_batch_size=experiences.shape[0])
    finns = [tf.ones((experiences.shape[0], 1,1))*actor.pad_value]
    unstacked_exp = tf.unstack(experiences, axis=1)
    for index in range(1,2*actor.dolinar_layers-2,2):
        finns.append(tf.reshape(unstacked_exp[index], (experiences.shape[0], 1,1)))
    final_preds = tf.concat(finns, axis=1)

    with tf.GradientTape() as tape:
        tape.watch(actor.trainable_variables)
        final_predss = actor(final_preds)
            
    da_dtheta1 = tape.gradient(final_predss, actor.trainable_variables, output_gradients=-dq_da)
    return da_dtheta1 


In [86]:
actor.lstm.stateful=False
actor.reset_states_workaround(new_batch_size=experiences.shape[0])


In [89]:

p1 = monster_gradient(actor, dq_da, experiences, optimizer_actor)
p2 = monster_gradient0(actor, dq_da, experiences, optimizer_actor)

In [90]:
p1

[<tf.Tensor: shape=(1, 1000), dtype=float32, numpy=
 array([[ 5.67119463e-10, -8.85343410e-10, -2.05919115e-10,
          3.42286754e-10,  5.44583378e-10, -3.10943077e-11,
         -2.75303547e-11,  1.09889298e-09,  3.14531456e-10,
         -2.67724514e-12, -4.50114807e-10, -9.67857905e-10,
         -5.25133973e-12,  4.52457988e-10, -1.48777532e-10,
         -1.95119199e-10,  1.10279585e-09, -7.78936970e-10,
         -7.75363662e-11,  5.95646699e-10,  3.07072062e-10,
         -2.98392283e-10,  5.07517695e-10, -2.94453420e-11,
         -4.27998713e-11, -1.16005427e-09,  1.65353176e-10,
          4.90927611e-11,  1.71218109e-10, -8.80522599e-11,
         -8.72051931e-10, -2.72250861e-10,  4.00835226e-10,
         -5.68753988e-10,  6.95558611e-10,  3.53939267e-10,
          5.15326781e-10,  2.22251245e-10, -1.03235212e-10,
          8.97226071e-10, -4.21958163e-11, -5.63887215e-10,
          6.11491080e-10, -1.16133314e-09,  1.42135522e-11,
         -3.94663385e-10,  3.48425705e-10,  1.89

In [102]:
p1[-2]/p2[-2]

<tf.Tensor: shape=(30, 1), dtype=float32, numpy=
array([[-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125],
       [-0.03125]], dtype=float32)>

In [103]:
1/32

0.03125

In [64]:
#@tf.function
def monster_gradient2(actor, dq_da, experiences, optimizer_actor):
    finns = [tf.ones((experiences.shape[0], 1,1))*actor.pad_value]
    unstacked_exp = tf.unstack(experiences, axis=1)
    for index in range(1,2*actor.dolinar_layers-2,2):
        finns.append(tf.reshape(unstacked_exp[index], (experiences.shape[0], 1,1)))
    final_preds = tf.concat(finns, axis=1)

    with tf.GradientTape() as gg:
        gg.watch(actor.trainable_variables)
        predictions = actor(final_preds)
            
    da_dtheta1 = predictions
    return gg.jacobian(da_dtheta1, actor.trainable_variables)
#tape.jacobian(da_dtheta1, actor.trainable_variables)
#https://github.com/tensorflow/tensorflow/issues/37053

In [65]:
actor.lstm.stateful=False
actor.reset_states_workaround(new_batch_size=experiences.shape[0])

print(monster_gradient2(actor, dq_da, experiences, optimizer_actor))
actor.reset_states_workaround(new_batch_size=int(1))
actor(context_outcome_actor)

[<tf.Tensor: shape=(32, 1, 1, 1, 1000), dtype=float32, numpy=
array([[[[[-1.2009596e-05,  1.8748457e-05,  4.3606419e-06, ...,
           -6.9060748e-06, -2.1024257e-06, -5.0446108e-08]]]],



       [[[[-1.2009596e-05,  1.8748457e-05,  4.3606419e-06, ...,
           -6.9060748e-06, -2.1024257e-06, -5.0446108e-08]]]],



       [[[[-1.2009596e-05,  1.8748457e-05,  4.3606419e-06, ...,
           -6.9060748e-06, -2.1024257e-06, -5.0446108e-08]]]],



       ...,



       [[[[-1.2009596e-05,  1.8748457e-05,  4.3606419e-06, ...,
           -6.9060748e-06, -2.1024257e-06, -5.0446108e-08]]]],



       [[[[-1.2009596e-05,  1.8748457e-05,  4.3606419e-06, ...,
           -6.9060748e-06, -2.1024257e-06, -5.0446108e-08]]]],



       [[[[-1.2009595e-05,  1.8748455e-05,  4.3606433e-06, ...,
           -6.9060757e-06, -2.1024280e-06, -5.0446125e-08]]]]],
      dtype=float32)>, <tf.Tensor: shape=(32, 1, 1, 250, 1000), dtype=float32, numpy=
array([[[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0.,

<tf.Tensor: shape=(1, 1, 1), dtype=float32, numpy=array([[[0.00268058]]], dtype=float32)>

In [82]:
finns = [tf.ones((experiences.shape[0], 1,1))*actor.pad_value]
unstacked_exp = tf.unstack(experiences, axis=1)
for index in range(1,2*actor.dolinar_layers-2,2):
    finns.append(tf.reshape(unstacked_exp[index], (experiences.shape[0], 1,1)))
final_preds = tf.concat(finns, axis=1)

with tf.GradientTape(persistent=True) as gg:
    gg.watch(actor.trainable_variables)
    predictions = actor(final_preds)
    pp=[]
    for k in tf.unstack(predictions):
        pp.append(gg.gradient(k, actor.trainable_variables))

In [ ]:
loss_ev = []
history_betas = []
total_episodes = 10**3
batch_size = 128.           

context_outcome_actor = np.reshape(np.array([actor.pad_value]),(1,1,1)).astype(np.float32)
history_betas.append(np.squeeze(actor(context_outcome_actor)[0]))


plt.figure(figsize=(30,30))


ax1 = plt.subplot2grid((2,2),(0,0))
ax2 = plt.subplot2grid((2,2),(0,1))
ax3 = plt.subplot2grid((2,2),(1,0))
ax4 = plt.subplot2grid((2,2),(1,1))

axes = {"0.0":ax1, "1.0":ax2}


for k in tqdm(range(10**3)):
    experiences = buffer.sample(batch_size).astype(np.float32)
    samples, zerolabs = critic_target.process_sequence_tf(experiences)
    labels_critic = critic_target.give_td_errors_tf( samples, zerolabs)
    loss_critic = step_critic_tf(samples ,labels_critic, critic, optimizer_critic)
    critic_target.update_target_parameters(critic)
    #loss_ev.append(loss_critic.numpy())
    #### actor ####
    
    actor.lstm.stateful=False
    actor.reset_states_workaround(new_batch_size=int(batch_size))
    
    dq_da = critic_grad_tf(critic, experiences)
    monster_gradient(actor, -dq_da, experiences, optimizer_actor)
    
    actor.reset_states_workaround(new_batch_size=1)

    context_outcome_actor = np.reshape(np.array([actor.pad_value]),(1,1,1)).astype(np.float32)
    history_betas.append(np.squeeze(actor(context_outcome_actor)[0]))
    
    if k%int(total_episodes/10)==1:
        plot(critic, episode=k, last_episode=False, max_episode = total_episodes)
plot(critic, episode=k, last_episode=True, history_betas=history_betas)    

Let0s analyze critic graddient

In [ ]:
optimizer_actor.learning_rate.assign(tf.Variable(0.01))

In [ ]:
actor = Actor(nature="primary")
actor(context_outcome_actor)

In [ ]:
batch_size=32
for k in tqdm(range(10**2)):
    experiences = buffer.sample(batch_size).astype(np.float32)
    
    actor.lstm.stateful=False
    actor.reset_states_workaround(new_batch_size=int(batch_size))
    
    dq_da = critic_grad_tf(critic, experiences)
    monster_gradient(actor, -dq_da, experiences, optimizer_actor)
    
    actor.reset_states_workaround(new_batch_size=1)

    context_outcome_actor = np.reshape(np.array([actor.pad_value]),(1,1,1)).astype(np.float32)
    history_betas.append(np.squeeze(actor(context_outcome_actor)[0]))
    

In [ ]:
plt.plot(history_betas)

In [ ]:
experiences.shape

In [ ]:
def critic_grad_tf(critic, experiences):
    with tf.GradientTape() as tape:
        unstacked_exp = tf.unstack(tf.convert_to_tensor(experiences), axis=1)
        to_stack = []
        actions_wathed_index = []
        for index in range(0,experiences.shape[-1]-3,2): # I consider from first outcome to last one (but guess)
            actions_wathed_index.append(index)
            to_stack.append(tf.reshape(unstacked_exp[index],(experiences.shape[0],1,1)))

        actions_indexed = tf.concat(to_stack,axis=1)
        tape.watch(actions_indexed)

        index_actions=0
        watched_exps=[tf.ones((experiences.shape[0],1,1))*critic.pad_value]
        watched_actions_unstacked = tf.unstack(actions_indexed, axis=1)
        for index in range(0,experiences.shape[-1]-1):
            if index in actions_wathed_index:
                watched_exps.append(tf.expand_dims(watched_actions_unstacked[index_actions], axis=2))
                index_actions+=1
            else:
                watched_exps.append(tf.reshape(unstacked_exp[index],(experiences.shape[0],1,1)))

        qvals = critic(tf.reshape(tf.concat(watched_exps, axis=2), (experiences.shape[0],critic.dolinar_layers+1,2)))

        dq_da = tape.gradient(qvals, actions_indexed)
        #tf.print("dq_da mean", tf.math.reduce_mean(dq_da))
        return dq_da


In [ ]:
critic_grad_tf(critic, experiences)

In [ ]:
experiences = experiences[:5]

with tf.GradientTape() as tape:
    unstacked_exp = tf.unstack(tf.convert_to_tensor(experiences), axis=1)
    to_stack = []
    actions_wathed_index = []
    for index in range(0,experiences.shape[-1]-3,2): # I consider from first outcome to last one (but guess)
        actions_wathed_index.append(index)
        to_stack.append(tf.reshape(unstacked_exp[index],(experiences.shape[0],1,1)))
    actions_indexed = tf.concat(to_stack,axis=1)
    tape.watch(actions_indexed)

In [ ]:
experiences

In [ ]:
actions_indexed

In [ ]:
experiences = experiences[:5]

with tf.GradientTape() as tape:
    unstacked_exp = tf.unstack(tf.convert_to_tensor(experiences), axis=1)
    to_stack = []
    actions_wathed_index = []
    for index in range(0,experiences.shape[-1]-3,2): # I consider from first outcome to last one (but guess)
        actions_wathed_index.append(index)
        to_stack.append(tf.reshape(unstacked_exp[index],(experiences.shape[0],1,1)))
    actions_indexed = tf.concat(to_stack,axis=1)
    tape.watch(actions_indexed)

    index_actions=0
    watched_exps=[tf.ones((experiences.shape[0],1,1))*critic.pad_value]
    watched_actions_unstacked = tf.unstack(actions_indexed, axis=1)
    for index in range(0,experiences.shape[-1]-1):
        if index in actions_wathed_index:
            watched_exps.append(tf.expand_dims(watched_actions_unstacked[index_actions], axis=2))
            index_actions+=1
        else:
            watched_exps.append(tf.reshape(unstacked_exp[index],(experiences.shape[0],1,1)))

    qvals = critic(tf.reshape(tf.concat(watched_exps, axis=2), (experiences.shape[0],critic.dolinar_layers+1,2)))

In [ ]:
qvals

In [ ]:
critic(critic.process_sequence(experiences)[0]) - qvals

In [ ]:
experiences = experiences[:5]

with tf.GradientTape() as tape:
    unstacked_exp = tf.unstack(tf.convert_to_tensor(experiences), axis=1)
    to_stack = []
    actions_wathed_index = []
    for index in range(0,experiences.shape[-1]-3,2): # I consider from first outcome to last one (but guess)
        actions_wathed_index.append(index)
        to_stack.append(tf.reshape(unstacked_exp[index],(experiences.shape[0],1,1)))
    actions_indexed = tf.concat(to_stack,axis=1)
    tape.watch(actions_indexed)

    index_actions=0
    watched_exps=[tf.ones((experiences.shape[0],1,1))*critic.pad_value]
    watched_actions_unstacked = tf.unstack(actions_indexed, axis=1)
    for index in range(0,experiences.shape[-1]-1):
        if index in actions_wathed_index:
            watched_exps.append(tf.expand_dims(watched_actions_unstacked[index_actions], axis=2))
            index_actions+=1
        else:
            watched_exps.append(tf.reshape(unstacked_exp[index],(experiences.shape[0],1,1)))

    qvals = critic(tf.reshape(tf.concat(watched_exps, axis=2), (experiences.shape[0],critic.dolinar_layers+1,2)))
    dq_da = tape.gradient(qvals, actions_indexed)
    

Vale, it makes what we want, \grad_a Q(s,a).

Let's look the last one

In [ ]:
for k in da_dtheta1:
    print(k.shape)

In [ ]:
optimizer_actor.apply_gradients(zip(grad_j, actor.trainable_variables))

In [ ]:
grad_j

Todo junto 

In [ ]:
actor.reset_states_workaround(new_batch_size=1)
actor(context_outcome_actor)

In [ ]:
#@tf.function
def give_gradients_actions(experiences, actor):

    with tf.GradientTape() as tape:
        tape.watch(actor.trainable_variables)
        finns = [tf.ones((experiences.shape[0], 1,1))*actor.pad_value]
        unstacked_exp = tf.unstack(experiences, axis=1)
        for index in range(1,2*actor.dolinar_layers-2,2):
            finns.append(tf.reshape(unstacked_exp[index], (experiences.shape[0], 1,1)))
        final_preds = tf.concat(finns, axis=1)
        finalss = actor(final_preds)
    return tape.jacobian(finalss, actor.trainable_variables)

def apply_grads_J(da_dtheta1, dq_da, actor, optimizer_actor):
    variables = actor.trainable_variables
    grad_j=[]
    for k in range(len(da_dtheta1)):
        onnns = tf.ones(len(tf.shape(da_dtheta1[k]))-1)
        multip = tf.reshape(dq_da,tf.cast(tf.concat([tf.Variable([batch_size]),onnns], axis=0), np.int32))*da_dtheta1[k]
        multip = tf.reshape(tf.math.reduce_mean(multip, axis=0), variables[k].shape)
        grad_j.append(multip)
    optimizer_actor.apply_gradients(zip(grad_j, actor.trainable_variables))
    return

def gradeame_J(experiences, actor, dq_da, optimizer_actor):
    gg = give_gradients_actions(experiences, actor)
    apply_grads_J(gg, dq_da, actor, optimizer_actor)
    return


gradeame_J(experiences, actor, dq_da, optimizer_actor)